In [7]:
import pyomo.environ as pyo

In [8]:
i = 0.05    #discount rate
MN = {1:250, 2:40, 3:20, 4:250, 5:10}    #maximum number available for each technology
sigma=0.0002
etta=0.95
CP = {1:614, 2:3200, 3:1713.15, 4:130, 5:2000}    #capital cost coefficient for each technology($)
CM = {1:0, 2:100, 3:0.2, 4:0, 5:0}    #capital cost coefficient for each technology($)

In [9]:
model = pyo.ConcreteModel()

model.T = pyo.RangeSet(8760) #24 hours in a year
model.C = pyo.RangeSet(5)  #no. of technologies; PV, Wind, Diesel Generator, Battery and Converter/inventer respectively.

model.P = pyo.Var(model.C, model.T, within=pyo.NonNegativeReals)       #power produced by each technology
#model.PVT = pyo.Var(model.T, within=pyo.NonNegativeReals)              #power produced by solar panles
#model.WT = pyo.Var(model.T, within=pyo.NonNegativeReals)               #power produced by wind turbine
#model.DGT = pyo.Var(model.T, within=pyo.NonNegativeReals)              #power produced by Diesel Generator 
#model.EBT = pyo.Var(model.T, within=pyo.NonNegativeReals)               #power produced by Battery storage
#model.PCONVT = pyo.Var(model.T, within=pyo.NonNegativeReals)            #power produced by converter/inverter

model.N = pyo.Var(model.C, within=pyo.Integers)       #number of each technology
#model.NPV = pyo.Var( within=pyo.NonNegativeReals)             #number of solar panles
#model.NW = pyo.Var(model.T, within=pyo.NonNegativeReals)              #number of wind turbine
#model.NDG = pyo.Var(model.T, within=pyo.NonNegativeReals)             #number of Diesel Generator
#model.NEB = pyo.Var(model.T, within=pyo.NonNegativeReals)              #number of Batteries
#model.NCONV = pyo.Var(model.T, within=pyo.NonNegativeReals)            #number of converter/inverter

#model.CB = pyo.Var(model.T, within=pyo.NonNegativeReals)       #cost coefficient of each Battery 
#model.CCONV = pyo.Var(model.T, within=pyo.NonNegativeReals)    #cost coefficient of each converter/inverter

model.sb = pyo.Var(model.T, within=pyo.Binary)           #binary for battery
model.load = pyo.Param(model.T, initialize=load)
model.PV = pyo.Param(model.T, initialize=PV)
model.W = pyo.Param(model.T, initialize=W)
#model.V = pyo.Param(model.T, initialize=Speed)
model.MN = pyo.Param(model.C, initialize=MN)
model.CP = pyo.Param(model.C, initialize=CP)
model.CM = pyo.Param(model.C, initialize=CM)


NameError: name 'PV' is not defined

In [4]:
#defining objective function
def obj_rule(model):
    return sum(((i*((i+1)**n))/(((i+1)**n)-1)) *(model.CF[c]*model.N[c]) for c in model.C for t in model.T) + sum(model.CM[c]*model.N[c] for c in model.C for t in model.T) + sum(1.248*(0.246*model.P[3,t]+0.0845*1.9)*model.N[3] for t in model.T)
model.obj=pyo.Objective(rule=obj_rule, sense=pyo.minimize)


In [6]:
#defining demand constraints
def Demand_rule(model,t):
    return sum(model.P[c,t]*D[c,g] for c in model.C ) == sum(model.load[t] for c in model.C)
model.con1=pyo.Constraint(model.T, rule=Demand_rule)

In [ ]:
#defining PV contraints
def PV_rule(model,t):
    return model.P[1,t] <= model.PV[t] * model.N[1]
model.con2=pyo.Constraint(model.T, rule=PV_rule)

In [ ]:
#defining WIND BOUNDRIESS
#def W_rule(model,t):
#    if model.V[t]<= 2.5 or model.V[t]>=13:
#        return model.W[t]==0
#    elif model.V[t]>2.5 and model.V[t]<11:
#        return model.W[t]=(model.V[t]-2.5)/(11-13)
#    else:
#        return model.W[t]==1

def W_rule(model,t):
    return model.P[2,t]<=model.W[t] * model.N[2]
model.con3=pyo.Constraint(model.T, rule=W_rule)

In [ ]:
#defining diesel
def Diesel_rule(model,t):
    if model.P[1,t]+model.P[2,t]+model.P[4,t]> model.load[t]:
        return model.P[3,t]==0
    else:
        return model.P[3,t]==model.load[t]-(model.P[1,t]+model.P[2,t]+model.P[4,t])
model.con4=pyo.Constraint(model.T, rule=Diesel_rule)

def diesel_bound(model,t):
    return model.P[3,t]<=1.9
model.con5=pyo.Constarint(model.T,rule=diesel_bound)

def number_diesel(model):
    return model.N[3]==1
model.con6=pyo.Constarint(rule=number_diesel)

In [ ]:
#defining battery
def battery_rule(model,t):
    if model.P[1,t]+model.P[2,t] > model.load[t]:
        return model.P[4,t]=model.P[4,-1]*(1-sigma) + (model.P[1,t]+model.P[2,t]-(model.load[t]/etta))
    else:
        return model.P[4,t]=model.O[4,t-1]*(1-sigma) -(model.P[1,t]+model.P[2,t]+(model.load[t]/etta))
model.con7=pyo.Constraint(model.T, rule=battery_rule)

In [ ]:
#defining boundries for number of each technology
def Number_rule(model,c):
    return model.N[c]<=model.MN[c]
model.con8=pyo.Constraint(model.C, rule=Number_rule)

In [ ]:
#defining miimum soc:
def soc_max(model,t):
    return model.P[4,1]<=1.35
model.con9=pyo.Constraint(model.T, rule=soc_max)

def soc_min(model,t):
    return model.P[4,1]>=1.08
model.con10=pyo.Constraint(model.T, rule=soc_min)


In [10]:
status = pyo.SolverFactory("mindtpy").solve(model, mip_solver='glpk', nlp_solver='ipopt')

pyo.assert_optimal_termination(status)
print(pyo.check_optimal_termination(status))



    functionality.  (deprecated in 6.4.2) (called from
    /Users/delarambayat/opt/anaconda3/lib/python3.9/site-
    packages/pyomo/contrib/mindtpy/initialization.py:154)
True


In [11]:
model.display()

Model unknown

  Variables:
    SU : Size=216, Index=SU_index
        Key        : Lower : Value              : Upper : Fixed : Stale : Domain
         (1, 1, 1) :     0 :  266.6666666666667 :  None : False : False : NonNegativeReals
         (1, 1, 2) :     0 :                0.0 :  None : False : False : NonNegativeReals
         (1, 1, 3) :     0 :                0.0 :  None : False : False : NonNegativeReals
         (1, 1, 4) :     0 :                0.0 :  None : False : False : NonNegativeReals
         (1, 1, 5) :     0 :                0.0 :  None : False : False : NonNegativeReals
         (1, 1, 6) :     0 :                0.0 :  None : False : False : NonNegativeReals
         (1, 1, 7) :     0 :                0.0 :  None : False : False : NonNegativeReals
         (1, 1, 8) :     0 :                0.0 :  None : False : False : NonNegativeReals
         (1, 1, 9) :     0 :                0.0 :  None : False : False : NonNegativeReals
        (1, 1, 10) :     0 :          

In [12]:
print(pyo.value(model.obj))

99715.45576854835
